## Data Load and Preprocessing

In [ ]:
import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import itertools

creds, _ = default()
gc = gspread.authorize(creds)

sheet_name = input("응답 시트 파일명 (해당 공유 폴더가 내 드라이브에 있어야함!): ")
worksheet = gc.open(sheet_name).sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows)
df.fillna("-")
df = df.iloc[:, -1][1:]
temp = df.str.split("\n")
temp = list(itertools.chain(*temp))

text = df.iloc[:-1][1:]

In [ ]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from krwordrank.sentence import summarize_with_sentences
import re

kkma = Kkma()
okt = Okt()

text = text.apply(lambda x: re.sub("\n", " ", x))
text = text.apply(lambda x: re.sub("\u200b", "", x))
text = text.apply(lambda x: re.sub("\xa0", "", x))
# text = text.apply(lambda x: re.sub("([a-zA-Z])", "", x))
text = text.apply(lambda x: re.sub("[ㄱ-ㅎㅏ-ㅣ]+", "", x))
text = text.apply(lambda x: re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", " ",x))

## Keyword Extraction and WordCloud

In [ ]:
for s in text:
    if len(s)==0: continue
    sentance = okt.pos(s, stem=True)
    word = []
    for i in sentance:
        if not (i[1]=="Noun" or  i[1]=="Alpha"): continue
        if len(i[0])==1: continue
        word.append(i[0])
    word = " ".join(word)

In [ ]:
from krwordrank.word import summarize_with_keywords

keywords = summarize_with_keywords(text, min_count=2, max_length=30, beta=0.85, max_iter=10, verbose=True)

In [ ]:
tab = 0
for _ in list(keywords.keys()):
    tab += 1
    print(_, end=" / ")
    if tab%25==0: print()


In [ ]:
passwords = {word:score for word, score in sorted(keywords.items(), key=lambda x:-x[1])[:300]}

from wordcloud import WordCloud
import matplotlib.pyplot as plt

krwordrank_cloud = WordCloud(
    font_path = "/content/drive/MyDrive/malgun.ttf",
    width = 500,
    height = 500,
    max_font_size = 250,
    scale = 1.8,
    background_color = "black"
).generate_from_frequencies(passwords)

# Output
plt.figure(figsize=(15, 15))
plt.imshow(krwordrank_cloud)
plt.axis("off")
plt.show()